In [1]:
import warnings 
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
import tensorflow as tf
import re
import os
import nltk
import datetime

from gensim.models.fasttext import FastText
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm_notebook

In [2]:
# 데이터 불러오기

raw_train = pd.read_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/data/train.csv')
raw_test = pd.read_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/data/test_x.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/data/sample_submission.csv')

In [3]:
train = raw_train.copy()
test = raw_test.copy()

In [4]:
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)


stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [5]:
train['text'] = train['text'].str.lower().apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].str.lower().apply(alpha_num).apply(remove_stopwords)

In [6]:
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values.reshape(-1, 1)
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879, 1)


In [7]:
max_length = 500
padding_type='post'
vocab_size= 20000

In [8]:
tokenizer = Tokenizer(oov_token='oov', num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

x_train = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
x_test = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(x_train.shape, x_test.shape)

(54879, 500) (19617, 500)


## 1. FastText 사용하기

In [9]:
!pip install fasttext

In [10]:
import fasttext
import fasttext.util

In [11]:
print(f"== LOAD fasttext START at {datetime.datetime.now()}")
ft = fasttext.load_model('/content/drive/MyDrive/FastText/cc.en.300.bin')
print(f"== LOAD fasttext   END at {datetime.datetime.now()}")

== LOAD fasttext START at 2020-11-30 01:41:35.509494
== LOAD fasttext   END at 2020-11-30 01:43:32.109275


In [12]:
embedding_dim = 300
embedding_matrix = np.zeros( (len(word_index)+1, embedding_dim) )

In [13]:
# 임베딩테이블 만들기
for word, idx in word_index.items():
    embedding_vector = ft.get_word_vector(word)
    if embedding_vector is not None :
        embedding_matrix[idx] = embedding_vector

In [14]:
# 임베딩 테이블
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.01219675,  0.00147353, -0.03663736, ...,  0.03829937,
        -0.00697157,  0.03466895],
       [ 0.11233243, -0.06369802, -0.04766245, ...,  0.09823206,
        -0.07316797, -0.02172678],
       ...,
       [-0.01149012,  0.01776954, -0.0071756 , ...,  0.01885659,
        -0.02117096, -0.01539714],
       [-0.01791752,  0.01672363, -0.00729322, ...,  0.00017261,
         0.00151998, -0.02690443],
       [-0.04956884,  0.02368064,  0.01324621, ..., -0.00153529,
         0.00691159,  0.03626099]])

In [15]:
target_col = 'author'
n_class = 5
seed = 42

In [16]:
import tensorflow.keras.backend as K
# Mish 활성화 함수
def mish(x):
    return x * K.tanh(K.softplus(x))

In [17]:
vocab_size

20000

In [21]:
def get_model() :

    from tensorflow.keras import Sequential
    from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Dropout, Flatten, MaxPool1D, GlobalAveragePooling1D, AveragePooling1D, Concatenate
    import tensorflow as tf

    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix[0:20000]], input_length=max_length))
    model.add(Conv1D(filters=50, kernel_size=3, padding='same', activation=mish, strides=1))
    model.add(GlobalAveragePooling1D())
    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(50, activation=mish))
    model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.003))

    return model


## 2. Hold Out

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train_holdout, X_test_holdout, y_train_holdout, y_test_holdout = train_test_split(x_train, y, test_size=0.2, random_state=123)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)

In [ ]:
model = get_model()
history = model.fit(X_train_holdout, y_train_holdout, 
                    epochs           = 20,
                    callbacks        = [es],
                    batch_size       = 32,
                    validation_data = (X_test_holdout, y_test_holdout))

## 3. K-Fold

In [19]:

# 5-Fold 시행
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

In [22]:
validation_pred = np.zeros((y.shape[0], n_class))
test_pred = np.zeros((test.shape[0], n_class))

i = 0
for train_idx, val_idx in tqdm_notebook(cv.split(x_train, y)):
    print("{}-Fold" .format(i+1))
    X_train = x_train[train_idx]
    y_train = y[train_idx]

    X_validation = x_train[val_idx]
    y_validation = y[val_idx]

    CNN = get_model()

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2, restore_best_weights=True)

    CNN.fit(X_train, y_train,
             epochs           = 20,
             callbacks        = [es],
             batch_size       = 32,
             validation_data  = (X_validation, y_validation))
    
    validation_pred[val_idx, :] = CNN.predict(X_validation)
    test_pred += (CNN.predict(x_test) / 5)
    print('')

    i += 1

1-Fold
Epoch 1/20
1372/1372 [==============================] - 94s 68ms/step - loss: 1.0985 - val_loss: 0.8070
Epoch 2/20
1372/1372 [==============================] - 95s 69ms/step - loss: 0.6730 - val_loss: 0.7070
Epoch 3/20
1372/1372 [==============================] - 94s 69ms/step - loss: 0.5150 - val_loss: 0.7049
Epoch 4/20
1372/1372 [==============================] - 94s 69ms/step - loss: 0.4341 - val_loss: 0.7468
Epoch 5/20
1372/1372 [==============================] - 95s 69ms/step - loss: 0.3798 - val_loss: 0.8166
Epoch 00005: early stopping

2-Fold
Epoch 1/20
1372/1372 [==============================] - 94s 69ms/step - loss: 1.0822 - val_loss: 0.8036
Epoch 2/20
1372/1372 [==============================] - 95s 70ms/step - loss: 0.6577 - val_loss: 0.7108
Epoch 3/20
1372/1372 [==============================] - 94s 69ms/step - loss: 0.5121 - val_loss: 0.7295
Epoch 4/20
1372/1372 [==============================] - 95s 69ms/step - loss: 0.4235 - val_loss: 0.7726
Epoch 00004: early st

In [ ]:
CNN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 300)          6000000   
_________________________________________________________________
dropout_3 (Dropout)          (None, 500, 300)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 50)           45050     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 25)               

In [1]:
from sklearn.metrics import log_loss, recall_score, confusion_matrix

In [ ]:
y_pred = np.argmax(validation_pred, axis=1)
y_real = y.reshape((1, -1))[0]

In [2]:
confusion_matrix(y_pred, y_real)

NameError: ignored

In [ ]:
# log_loss

log_loss(pd.get_dummies(y_real), validation_pred)

In [ ]:
# 결과 저장
sample_submission[['0','1','2','3','4']] = test_pred
sample_submission.to_csv('/content/drive/MyDrive/[데이콘] 소설 작가 분류 AI 경진대회/02_나의코드/01_딥러닝/Test예측결과/submission_11.csv', index = False, encoding = 'utf-8')